In [67]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from patsy import dmatrices

In [88]:
credit = pd.read_csv('Credit.csv', usecols=list(range(1,12)))
credit_cp=credit.copy(deep=True)
credit_cp['Student_dummy'] = credit_cp.Student.map({'No':0, 'Yes':1})
credit_cp['Married_dummy'] = credit_cp.Married.map({'No':0, 'Yes':1})
credit_cp['Active'] = np.where(credit_cp['Balance']>0, 1,0)
credit_cp['HasCard']=np.where(credit_cp['Cards']>0, 1,0)
dummy=pd.get_dummies(credit_cp['Gender'])
credit_cp=pd.concat([credit_cp, dummy], axis=1)

In [89]:
credit_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 17 columns):
Income           400 non-null float64
Limit            400 non-null int64
Rating           400 non-null int64
Cards            400 non-null int64
Age              400 non-null int64
Education        400 non-null int64
Gender           400 non-null object
Student          400 non-null object
Married          400 non-null object
Ethnicity        400 non-null object
Balance          400 non-null int64
Student_dummy    400 non-null int64
Married_dummy    400 non-null int64
Active           400 non-null int32
HasCard          400 non-null int32
 Male            400 non-null uint8
Female           400 non-null uint8
dtypes: float64(1), int32(2), int64(8), object(4), uint8(2)
memory usage: 44.7+ KB


In [90]:
X=credit_cp.iloc[:, [2,4,13,14]].values
y=credit_cp.iloc[:,0].values

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=42)

In [91]:
model_xgb=xgb.XGBRegressor(booster='gbtree')
model_xgb.fit(X_train, y_train)

[19:12:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [92]:
model_xgb.score(X_test, y_test)

0.7154182726883911

In [93]:
%store -r estSalary

In [94]:
y_pred=model_xgb.predict(estSalary)
y_pred=y_pred[0]*10000
estSalary

[321, 18, 0, 0]

In [96]:
y_pred

142438.6978149414

In [95]:
%store y_pred

Stored 'y_pred' (float64)


In [97]:
%store -r estCreditScore

In [98]:
estCreditScore=np.asarray(estCreditScore)
estCreditScore=estCreditScore.reshape(1,3)

In [99]:
estCreditScore=pd.DataFrame(estCreditScore, columns=['Age', 'IsActiveMember','EstimatedSalary'])

In [100]:
banki=pd.read_csv('Dane_bank_nowe.csv', sep=',')
banki = banki.drop(["Unnamed: 0", 'RowNumber'], axis=1)

def div_2(x):
    if x >= 640:
        return 0
    else:
        return 1

banki['CreditScore_2']=banki.apply(lambda x: div_2(x['CreditScore']),axis=1 )

XX=banki.iloc[:, [5,10,11]]
yy=banki['CreditScore_2']

X_train, X_test, y_train, y_test=train_test_split(XX,yy, test_size=0.3, random_state=42, stratify=yy)
model2=xgb.XGBClassifier()
model2.fit(X_train, y_train)

y_pred2=model2.predict(estCreditScore)
y_pred2=y_pred2[0]

In [101]:
if y_pred2==1:
    info='Spłaci kredyt'
elif y_pred2==0:
    info='Nie spłaci kredytu'
else:
    info='Błąd'

In [102]:
%store info

Stored 'info' (str)
